In [78]:
import os
import torch
from sklearn import metrics
import logging
from timeit import default_timer as timer
import pandas as pn

In [2]:
from context import resnet
from context import utils
from context import data_loader
from context import data_utils

In [3]:
BATCH_SIZE = 2
STATE_DICT_PATH = '../state'
DATA_PATH = '../data/imagenet_red'
STORE_RESULT_PATH = ''

In [4]:
def evaluateModel(model, loader, classes, batch_size): 
    pred, grndT = [], []
    for (images, labels) in iter(loader):
        if torch.cuda.is_available():
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        pred = pred + [classes[predicted[k]] for k in range(min(batch_size, labels.shape[0]))]
        grndT = grndT + [classes[labels[j]] for j in range(min(batch_size, labels.shape[0]))]
    return grndT, pred

def timed_decorator(function):
    def inner(*args, **kwargs):
        start = timer()
        result = function(*args, **kwargs)
        end = timer()
        print(f'{(end - start) * 1000} [ms]')
        return result
    return inner

@timed_decorator
def executeQualityBench(arch_name: str, loader, skip_n: int, classes, batch_size: int):
    model = utils.getModelWithOptimized(arch_name, n=skip_n, batch_size=batch_size)

    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model).cuda()
    model.eval()
    arch = arch_name.split('-')[0]
    model, _, _, _ = utils.resumeFromPath(os.path.join(STATE_DICT_PATH, f'{arch}_model_best.pth.tar'), model)
    grndT, pred = evaluateModel(model, loader, classes, batch_size)

    #logging.debug(metrics.classification_report(grndT, pred, digits=3))
    #acc = metrics.accuracy_score(grndT, pred)
    #prec = metrics.precision_score(grndT, pred, average='macro')
    #rec = metrics.recall_score(grndT, pred, average='macro')
    #f1 = metrics.f1_score(grndT, pred, average='macro')
    return grndT, pred

In [5]:
experiment_data = list()

logging.basicConfig(level=logging.INFO)

model_name = 'resnet50-drop-rand-n'

skip_layers = 4
classes = data_utils.getLabelToClassMapping(DATA_PATH)
_, _, loader = data_loader.get_zipped_dataloaders(DATA_PATH, batch_size=BATCH_SIZE, use_valid=True)

../data/imagenet_red/index-val.txt
../data/imagenet_red/index-train.txt
../data/imagenet_red/index-val.txt


In [ ]:
for run in range(0, 30):
    print(f'Executing Run {run + 1} of 30')
    model = utils.getModelWithOptimized(model_name, n=skip_layers, batch_size=BATCH_SIZE)
    groundT, pred = executeQualityBench(model_name, loader, skip_n=skip_layers, classes=classes, batch_size=BATCH_SIZE)
    policy = resnet.DropPolicies.getDropPolicy()
    layer_config = []
    if isinstance(policy, resnet.DropPolicies.ResNetDropRandNPolicy):
        layer_config = policy.getDropConfig()
    experiment_data.append({
        'skip_n': skip_layers,
        'ground_truth': groundT, 
        'prediction': pred, 
        'configuration': layer_config,
        'acc': metrics.accuracy_score(groundT, pred)
        })

df = pn.DataFrame(experiment_data)
df.to_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'), index=False)

def print_data(data):
    for datum in data:
        print()
        print(datum['skip_n'])
        print(len(datum['ground_truth']))
        print(len(datum['prediction']))
        print(len(datum['configuration']))
        print()

In [8]:
temp = list()

temp.append({
    'skip_n': 1,
    'ground_truth': ['test2', 'test3', 'test3'], 
    'prediction': ['test1', 'test2', 'test3'], 
    'configuration': [True, False, False, True],
    'acc': 0.76
})
temp.append({
    'skip_n': 3,
    'ground_truth': ['test1', 'test1', 'test1'], 
    'prediction': ['test2', 'test2', 'test3'], 
    'configuration': [True, True, False, True],
    'acc': 0.70
})
temp.append({
    'skip_n': 1,
    'ground_truth': ['test3', 'test1', 'test3'], 
    'prediction': ['test1', 'test1', 'test3'], 
    'configuration': [False, False, False, False],
    'acc': 0.22
})


df1 = pn.DataFrame(temp)
df1

,skip_n,ground_truth,prediction,configuration,acc
0,1,"[test2, test3, test3]","[test1, test2, test3]","[True, False, False, True]",0.76
1,3,"[test1, test1, test1]","[test2, test2, test3]","[True, True, False, True]",0.70
2,1,"[test3, test1, test3]","[test1, test1, test3]","[False, False, False, False]",0.22


In [90]:
df = pn.read_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'))
df2 = pn.read_csv(os.path.join(STORE_RESULT_PATH, '0_resnet50_experiment_results.csv'))
df2
df = df.append(df2, ignore_index=True)
df

,skip_n,ground_truth,prediction,configuration,acc
0,1,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['cabbage butterfly', 'gorse', 'gorse', 'gorse...",[False False False False False False False Fal...,0.570825
1,1,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['roseate spoonbill', 'dishrag', 'gorse', 'doi...",[False False False False False False False Fal...,0.420719
2,1,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['angiosperm', 'tinfoil', 'gorse', 'gorse', 'g...",[False False False False True False False Fal...,0.591966
3,1,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['angiosperm', 'tinfoil', 'gorse', 'gorse', 'g...",[False False False False False False False Fal...,0.501057
4,1,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['wild calla', 'tinfoil', 'gorse', 'spice', 'g...",[False False False False False False True Fal...,0.547569
...,...,...,...,...,...
176,6,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['gazania', 'chainlink fence', 'gazania', 'cha...",[False True False True False False False Fal...,0.046512
177,6,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['paper', 'doily', 'paper', 'dishrag', 'chainl...",[ True False True False False True True Fal...,0.065539
178,6,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['roseate spoonbill', 'doily', 'roseate spoonb...",[False False True True True True True Fal...,0.067653
179,6,"['gorse', 'gorse', 'gorse', 'gorse', 'gorse', ...","['eggplant', 'gazania', 'gazania', 'gazania', ...",[ True False False True True False False Fal...,0.046512


In [91]:
def convertStringToBoolArray(string: str):
    if type(string) == list:
        return string
    return [item.replace('[', '').replace(']', '').strip() for item in string.split(' ')]

def convertStringToStringArray(string: str):
    return [
            x.replace('[', '')
            .replace(']', '')
            .replace("'", '')
            .strip() 
            for x in string.split(', ')
        ]


df['configuration'] = df['configuration'].apply(lambda item: convertStringToBoolArray(item))
df['ground_truth'] = df['ground_truth'].apply(lambda item: convertStringToStringArray(item))
df['prediction'] = df['prediction'].apply(lambda item: convertStringToStringArray(item))

def getTrueIndexList(bool_list):
    idx_ls = []
    for idx, val in enumerate(bool_list):
        if val:
            idx_ls.append(idx)
    return idx_ls

for n in df.skip_n.unique():
    entry = df[df['skip_n'] == n].max()
    grndT = entry.ground_truth
    pred = entry.prediction
    report_dict = metrics.classification_report(grndT, pred, digits=3,output_dict=True)
    logging.info(metrics.classification_report(grndT, pred, digits=3))
    
    temp_df = pn.DataFrame(report_dict).transpose()

    temp_df.to_csv(f'{n}-classification-report.csv', index=False)


INFO:root:                       precision    recall  f1-score   support

    Tulipa gesneriana      0.667     0.667     0.667        12
           angiosperm      0.250     0.083     0.125        12
                aster      0.667     0.500     0.571        12
                 beer      0.435     0.833     0.571        12
          bell pepper      0.500     0.455     0.476        11
                berry      0.000     0.000     0.000        12
    cabbage butterfly      0.667     0.833     0.741        12
             caryatid      1.000     0.583     0.737        12
      chainlink fence      0.750     0.750     0.750        12
              dishrag      0.778     0.583     0.667        12
                doily      0.700     0.583     0.636        12
             eggplant      0.400     0.545     0.462        11
               floret      0.250     0.083     0.125        12
       flower cluster      1.000     0.167     0.286        12
              gazania      0.857     0.600  

In [87]:
print(df.skip_n.unique())

[1 2 3 4 5 6]
